In [1]:
import json
from transformers import BertTokenizer
with open('./data_dir/train.json','r',encoding='utf-8') as f:
    a=json.load(f)
tokenizer=BertTokenizer.from_pretrained('./vocab/vocab.txt')
conten_tokens=tokenizer.tokenize(a[0]['content'].replace(' ','[Space]'))
conten_tokens

d:\Anaconda3\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:1643: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


['近',
 '期',
 '，',
 '美',
 '国',
 '国',
 '会',
 '众',
 '院',
 '通',
 '过',
 '法',
 '案',
 '，',
 '重',
 '申',
 '美',
 '国',
 '对',
 '台',
 '湾',
 '的',
 '承',
 '诺',
 '。',
 '对',
 '此',
 '，',
 '中',
 '国',
 '外',
 '交',
 '部',
 '发',
 '言',
 '人',
 '表',
 '示',
 '，',
 '有',
 '关',
 '法',
 '案',
 '严',
 '重',
 '违',
 '反',
 '一',
 '个',
 '中',
 '国',
 '原',
 '则',
 '和',
 '中',
 '美',
 '三',
 '个',
 '联',
 '合',
 '公',
 '报',
 '规',
 '定',
 '，',
 '粗',
 '暴',
 '干',
 '涉',
 '中',
 '国',
 '内',
 '政',
 '，',
 '中',
 '方',
 '对',
 '此',
 '坚',
 '决',
 '反',
 '对',
 '并',
 '已',
 '向',
 '美',
 '方',
 '提',
 '出',
 '严',
 '正',
 '交',
 '涉',
 '。',
 '事',
 '实',
 '上',
 '，',
 '中',
 '国',
 '高',
 '度',
 '关',
 '注',
 '美',
 '国',
 '国',
 '内',
 '打',
 '[UNK]',
 '台',
 '湾',
 '牌',
 '[UNK]',
 '、',
 '挑',
 '战',
 '一',
 '中',
 '原',
 '则',
 '的',
 '危',
 '险',
 '动',
 '向',
 '。',
 '近',
 '年',
 '来',
 '，',
 '作',
 '为',
 '[UNK]',
 '亲',
 '台',
 '[UNK]',
 '势',
 '力',
 '大',
 '本',
 '营',
 '的',
 '美',
 '国',
 '国',
 '会',
 '动',
 '作',
 '不',
 '断',
 '，',
 '先',
 '后',
 '通',
 '过',
 '[UNK]',
 '与',
 '台',
 '湾',
 '交',
 '往',
 '法',
 '

In [102]:
n=[]
import torch
# for i in a:
#     n.append(len(i['title']))

for i in a:
    n.append(len(i['content']))

# title中都很干净，content中有多余空格和换行符。是否去空格和换行符
# 标题平均长度13-14,最大长度40,最小6。取标题最大长度多少？>25的只有19个，大于20的只有205个
# 内容平均长度1292,最大9628,最小187,小于1000的只有1/4的样子,但是GPTchinese最大长度是1024，怎样减少数据丢失呢？？
    # 一种方案。。。
# '[Space]'是干啥的

# min(n)
n=torch.tensor(n,dtype=torch.long)
n[n<=1024].shape,n.shape

(torch.Size([1469]), torch.Size([5850]))

In [64]:
tokenizer.decode([tokenizer.pad_token_id])

'[PAD]'

In [65]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [9]:
encoded=tokenizer.encode_plus(a[0]['content'],a[0]['title'],max_length=1024,truncation='only_first',padding='max_length',return_token_type_ids=True,return_tensors='pt')

In [10]:
encoded['input_ids'],encoded['token_type_ids'],encoded['attention_mask'].shape[1]

(tensor([[ 101, 6818, 3309,  ..., 6603, 1300,  102]]),
 tensor([[0, 0, 0,  ..., 1, 1, 1]]),
 1024)

In [11]:
title_id=1
mask=(encoded['token_type_ids']==title_id).long() 
mask

tensor([[0, 0, 0,  ..., 1, 1, 1]])

In [35]:
from data_set import GPT2Dataset
from torch.utils.data import DataLoader
# self,tokenizer,max_len,title_max_len,data_dir,data_set_name,path_file=None,is_overwrite=False):
      
dataset=GPT2Dataset(tokenizer=tokenizer,max_len=1024,path_file='./data_dir/dev.json',title_max_len=0,data_dir='',is_overwrite=False,data_set_name='')
dataloader=DataLoader(dataset,batch_size=4)
for idx,batch in enumerate(dataloader):
    print(batch['input_ids'].shape)
    print(batch['token_type_ids'])
    break

iter: 100%|██████████| 1679/1679 [00:22<00:00, 74.93it/s]


torch.Size([4, 1, 1024])
tensor([[[0, 0, 0,  ..., 1, 1, 1]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 1, 1, 1]],

        [[0, 0, 0,  ..., 1, 1, 1]]])


In [60]:
labels=batch['input_ids'].contiguous()
labels.shape

torch.Size([4, 1, 1024])

In [46]:
labels[...,:-1,:].contiguous()

tensor([], size=(4, 0, 1024), dtype=torch.int64)